In [1]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

In [2]:
API_KEY = '588b5e772855b570d82d148878871433'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

In [3]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    if data['cod'] != 200:
      print(f"Error: {data['message']}")
      return None
    return {
        'city' : data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min' : round(data['main']['temp_min']),
        'temp_max' : round(data['main']['temp_max']),
        'description' : data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'Wind_Gust_Speed': data['wind']['speed'],
        'humidity': data['main']['humidity']
    }

In [4]:
def read_historical_data(filename):
    df = pd.read_csv(filename)
    df = df.dropna()
    df = df.drop_duplicates()
    return df

In [5]:
def prepare_data(data):
    le = LabelEncoder()
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])
    X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]
    Y = data['RainTomorrow']
    return X, Y, le

In [6]:
def train_rain_model(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)

    print("Mean Squared Error for Rain Model")
    print(mean_squared_error(Y_test, Y_pred))
    return model

In [7]:
def prepare_regression_data(data, feature):
    X, Y = [], []
    for i in range(len(data) -1):
        X.append(data[feature].iloc[i])

        Y.append(data[feature].iloc[i+1])
    X = np.array(X).reshape(-1,1)
    Y = np.array(Y)
    return X, Y

In [8]:
def train_regression_model(X, Y):
    model = RandomForestRegressor(n_estimators=100, random_state=41)
    model.fit(X, Y)
    return model

In [9]:
def predict_future(model, current_value):
    predictions = [current_value]
    for i in range(5):
        next_value = model.predict(np.array([[predictions[-1]]]))
        predictions.append(next_value[0])
    return predictions[1:]

In [10]:
def weather_view():
    city = input('Enter any city name:')
    current_weather = get_current_weather(city)

    if current_weather is None:
      return

    historical_data = read_historical_data('weather.csv')
    if historical_data is None or historical_data.empty:
      print("Error: Could not load historical data or it is empty.")
      return

    X, Y, le = prepare_data(historical_data)
    rain_model = train_rain_model(X, Y)
    wind_deg = current_weather['wind_gust_dir']%360
    compass_points = [
        ("N", 0,11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
    ]
    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)[0] # Get the compass point name
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather['Wind_Gust_Speed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp'],
    }

    current_df = pd.DataFrame([current_data])
    rain_prediction = rain_model.predict(current_df)[0]
    X_temp, Y_temp = prepare_regression_data(historical_data, 'Temp')
    X_hum, Y_hum = prepare_regression_data(historical_data, 'Humidity')
    temp_model = train_regression_model(X_temp, Y_temp)
    hum_model = train_regression_model(X_hum, Y_hum)
    future_temp = predict_future(temp_model, current_weather['current_temp'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])
    timezone = pytz.timezone('asia/Kolkata')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    print(f"city: {city}, {current_weather['country']}")
    print(f"current Temprature: {current_weather['current_temp']}")
    print(f"Feels Like: {current_weather['feels_like']}")
    print(f"Minimum Temprature: {current_weather['temp_min']}°C")
    print(f"Maximum Temprature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")
    print("\nFuture Temprature Predictions:")

    for time, temp in zip(future_times, future_temp):
      print(f"{time}: {round(temp, 1)}°C")
    print("\nFuture Humidity Predictions:")
    for time, humidity in zip(future_times, future_humidity):
      print(f"{time}: {round(humidity, 1)}%")


In [11]:
weather_view()

Enter any city name: ranchi


Mean Squared Error for Rain Model
0.1506849315068493
city: ranchi, IN
current Temprature: 22
Feels Like: 23
Minimum Temprature: 22°C
Maximum Temprature: 22°C
Humidity: 94%
Weather Prediction: mist
Rain Prediction: Yes

Future Temprature Predictions:
07:00: 23.1°C
08:00: 17.0°C
09:00: 21.0°C
10:00: 19.0°C
11:00: 24.3°C

Future Humidity Predictions:
07:00: 60.8%
08:00: 58.2%
09:00: 54.9%
10:00: 57.1%
11:00: 39.6%
